# Modélisation avec PyCaret – Bank Marketing

Ce notebook utilise la bibliothèque PyCaret pour créer un modèle de classification visant à prédire si un client va souscrire à un dépôt à terme ou non (`y`).


## Importation des modules et des données nettoyées

In [ ]:
from pycaret.classification import *
import pandas as pd


# Recharger les données (ou utiliser le df_clean du notebook précédent si tu le lies)
df = pd.read_csv("data/bank_marketing/bank/bank.csv", sep=';')

# Nettoyage rapide : remplacer 'unknown' par NaN
df.replace('unknown', pd.NA, inplace=True)

# Supprimer les lignes incomplètes
df = df.dropna()

# Vérifions le type de la cible
df['y'].value_counts()

## Initialisation de l’environnement PyCaret

In [ ]:
clf = setup(
    data=df,
    target='y',
    session_id=42,
    verbose=False,
    categorical_features=['job', 'marital', 'education', 'default',
                          'housing', 'loan', 'contact', 'month', 'poutcome']
)

Lorsque tu exécutes setup(), PyCaret fait automatiquement :

La détection des types de colonnes (numérique, texte, booléen…)

Le prétraitement :

Encodage des colonnes catégorielles

Suppression/Imputation des valeurs manquantes

Séparation du jeu d'entraînement et de test

L'affichage du résumé des données traitées

La sauvegarde de cette "expérience" pour les étapes suivantes



## Comparaison automatique des modèles

PyCaret va tester plein de modèles (logistic regression, decision tree, random forest, etc.) et afficher leurs performances, en particulier l’accuracy.

In [ ]:
# Comparer les modèles automatiquement
best_model = compare_models()

## Ce que fait le programme :
Entraîne tous les modèles de classification disponibles

Affiche un tableau de comparaison avec des métriques comme :

Accuracy (précision globale)

AUC (performance sur les classes)

Recall, Precision, etc.

Sauvegarde le meilleur modèle dans la variable best_model

## Résultats de la comparaison

Le tableau affiche les performances de tous les modèles testés automatiquement par PyCaret.  
Le modèle ayant la **meilleure accuracy** est sélectionné.  
Cela signifie qu'il prédit correctement la plus grande proportion de "yes" ou "no".


## Affichage du meilleur modèle

In [ ]:
# Afficher les détails du meilleur modèle
print(best_model)

## Visualiser les métriques du modèle

### Objectif : visualiser les erreurs du modèle

In [ ]:
plot_model(best_model, plot='confusion_matrix')

Un tableau 2x2 qui compare les vraies réponses avec les prédictions du modèle :

|                                          | Prédit "no"           | Prédit "yes"          |
| ---------------------------------------- | --------------------- | --------------------- |
| **Vrai "no"** (clients qui ont refusé)   |  Vrais négatifs (TN) |  Faux positifs (FP)  |
| **Vrai "yes"** (clients qui ont accepté) |  Faux négatifs (FN)  |  Vrais positifs (TP) |


Interprétation :
- Les "vrais positifs" = bien prédits comme "yes"

- Les "faux négatifs" = le modèle a dit "no" alors que c’était "yes" → mauvais pour une campagne marketing !

Le modèle est meilleur s’il a :

peu de FN (ne pas rater des clients intéressés)

un bon équilibre entre TP et TN


### Objectif : mesurer la capacité de séparation entre "yes" et "no"

In [ ]:
plot_model(best_model, plot='auc') 

C’est la courbe ROC (Receiver Operating Characteristic). L’AUC (area under the curve) donne une note de 0 à 1 :

 AUC = 1.0 = parfait

 AUC > 0.8 = très bon

 AUC ≈ 0.5 = modèle aléatoire

### interprétation :

- Si la courbe est en haut à gauche, le modèle sépare bien les clients intéressés des non intéressés.

- L’AUC est moins influencée par les déséquilibres de classes que l’accuracy.


### Objectif : voir quelles variables influencent le plus les prédictions

In [ ]:
plot_model(best_model, plot='feature')

### Analyse des variables importantes (Feature Importance)

Le graphique montre que la variable `duration` est la plus influente dans les décisions du modèle.  
Cela signifie que les clients ayant eu de longs appels sont plus susceptibles de souscrire à l’offre.  
Les résultats des campagnes précédentes (`poutcome`) et le canal de communication (`contact`) sont aussi très prédictifs.

En revanche, des variables comme l’âge ou le niveau d’éducation ont un impact plus faible dans ce modèle.


### Objectif: voir comment le modèle apprend à mesure qu'on lui donne plus de données

In [ ]:
plot_model(best_model, plot='learning')

### Analyse de la courbe d'apprentissage

Le graphe montre que la performance du modèle augmente avec plus de données, puis se stabilise.  
Les courbes d'entraînement et de validation sont proches, ce qui indique une bonne généralisation.  
Cela signifie que le modèle n’est pas en situation de surapprentissage, et qu’il est capable de faire de bonnes prédictions sur de nouvelles données.


## Prédictions sur de nouvelles données

In [ ]:
# Créer un modèle final
final_model = finalize_model(best_model)

# Exemple de prédiction sur quelques lignes
predictions = predict_model(final_model, data=df.sample(5))
predictions

## Conclusion

Grâce à PyCaret, nous avons :
- testé plusieurs modèles de classification,
- sélectionné automatiquement le modèle avec la meilleure performance,
- évalué sa précision (accuracy) et sa robustesse.

Ce modèle peut maintenant être utilisé pour prédire si de nouveaux clients vont souscrire à une offre ou non.
